<a href="https://colab.research.google.com/github/HJJunn/K-Project/blob/main/K%ED%94%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#데이터 불러오기
df = pd.read_csv("/content/drive/MyDrive/k프로젝트 추천시스템.csv")
df

In [ ]:
#데이터 확인하기
df.info()

In [ ]:
df.describe().T

In [ ]:
df.columns

In [ ]:
df.StudentID.nunique()

In [ ]:
df1 = df.copy()

In [ ]:
#사용자간의 유사도 찾기 - 코사인 유사도
from sklearn.metrics.pairwise import cosine_similarity
def similar_users(id, matrix, k=3):
    user = matrix(matrix.index == id)
    other_users = matirx(matrix.index != id)
    s = cosine_similarity(user,other_users)[0].tolist()


In [ ]:
#로지스틱 회귀분석하기
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

# 데이터 전처리  Min-Max 스케일링
scaler = MinMaxScaler()
df[['Clicks', 'Searches', 'Duration']] = scaler.fit_transform(df[['Clicks', 'Searches', 'Duration']])
print(df)
#독립변수
X = df[['Clicks', 'Likes','Searches', 'Duration']]
#종속변수
y = df['Visits']
#데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 로지스틱 회귀 모델 생성
model = LogisticRegression()

# 모델 학습
model.fit(X_train, y_train)

# 테스트 데이터로 예측
y_pred = model.predict(X_test)

# 회귀 계수 출력
coef = model.coef_[0]
for i, coeff in enumerate(coef):
    print(f"독립 변수 {X.columns[i]}의 회귀 계수: {coeff:.2f}")

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 예시: VIF 계산
X = df[['Clicks', 'Likes','Searches', 'Duration']]
vif_data = pd.DataFrame()
vif_data['독립변수'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# 결과 출력
print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

In [ ]:
corr = df[['Clicks', 'Likes','Searches', 'Duration','Visits']].corr()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize = (10,9))
sns.heatmap(corr, annot = True, fmt = '.2f')

In [ ]:
#회귀 계수를 가중치로 하여 선호도 변수 생성
df1['Preference'] = np.where(
    (df1['Clicks'] >= 4) & (df1['Searches'] >= 3) & (df1['Duration'] >= 150),
    df1['Clicks'] * coef[0] +
    df1['Likes'] * coef[1] +
    df1['Duration'] * coef[2] +
    df1['Searches'] * coef[3],
    0
)
df1

In [ ]:
df1['Preference'] = df1.Preference.round()/10

In [ ]:
df1.describe()

In [ ]:
df_visits = df1[df1['Visits'] == 0]
df_visits

In [ ]:
df_pre= df1[df1['Preference'] == 0]
df_pre

In [ ]:
#피벗테이블로 변경
pre_matrix = df1.pivot_table(index = 'Booth', columns = 'StudentID', values = 'Preference')
pre_matrix

In [ ]:
pre_matrix_T = pre_matrix.T
pre_matrix_T.head()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_pre = cosine_similarity(pre_matrix, pre_matrix)
similarity_pre

In [ ]:
simi_pre = pd.DataFrame(
    data = similarity_pre,
    index = pre_matrix.index,
    columns = pre_matrix.index
)
simi_pre.head()

In [ ]:
#가장 유사도가 높은 top5
def recommand_Booth(Booth):
    return simi_pre[Booth].sort_values(ascending = False)[:6]

In [ ]:
recommand_Booth("공연1")

In [ ]:
# 사용자가 평가하지 않은 아이템 찾기 함수
def find_unrated_items(user_id):
    return pre_matrix.index[pre_matrix.loc[:, user_id] == 0]

# 사용자가 평가하지 않은 아이템에 대해 선호도 예측 함수
def predict_preference(user_id):
    unrated_items = find_unrated_items(user_id)
    predicted_preferences = {}

    for item in unrated_items:
        # 해당 아이템의 유사도가 높은 다른 아이템들을 가져옴
        similar_items = simi_pre[item].drop(item)

        # 다른 아이템들의 평점과 유사도를 곱하여 가중 평점 계산
        weighted_sum = 0
        similarity_sum = 0

        for similar_item, similarity in similar_items.items():
            rating = pre_matrix.loc[similar_item, user_id]  # 사용자가 평가한 해당 아이템의 평점
            if rating != 0:  # 사용자가 평가한 아이템일 경우에만 가중 평점에 포함
                weighted_sum += rating * similarity
                similarity_sum += similarity

        if similarity_sum != 0:
            predicted_preferences[item] = weighted_sum / similarity_sum
        else:
            predicted_preferences[item] = 1  # 모든 유사도가 0일 경우 0으로 예측

    return predicted_preferences


user_id = 202301244
predicted_preferences = predict_preference(user_id)
print(f"{user_id}에 대한 아이템별 선호도 예측:")
for item, preference in predicted_preferences.items():
    print(f"{item}: {preference:.2f}")

In [ ]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):

    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 인접 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))